## Neighbours

In [ ]:
class Tuile:
    def __init__(self, action):
        # veg : price, productivity, capacity
        self.SCORE = {1:[10,1,1], 2:[8,1,1], 3:[5,1,1], 4:[2,1,1], 5:[6,1,1]}, 
        self.NEMESIS = {1:[], 2:[1,5], 3:[], 4:[3], 5:[]}
        self.rewards = {'PENALTY_NEMESIS':-5}

In [ ]:
    def affinity(self): 

        x = action_[0]
        y = action_[1]
        
        if y-1 >=0:
            north = self._state[x, y-1]
        north = None
        if x+1 >=0 and y-1 >=0:
            north_east = self._state[x+1, y-1]
        north_east = None
        if x+1 >=0:
            east = self._state[x+1, y]
        east = None
        if x+1 >=0 and y+1 >=0:
            south_east = self._state[x+1, y+1]
        south_east = None
        if y+1 >= 0:
            south = self._state[x, y+1]
        south = None
        if x-1 >=0 and y+1 >=0:
            south_west = self._state[x-1, y+1]  
        south_west = None
        if x-1 >=0:
            west = self._state[x-1, y]
        west = None
        if x-1 >=0 and y-1 >=0:
            north_west = self._state[x-1, y-1]
        north_west = None
                                          
        for nem in self.NEMESIS[action_[2]]:
            if nem in [north, north_east, east, south_east,
                       south, south_west, west, north_west]:
                reward = self.rewards['penalty_nemesis']
        
    def revenue(self):
        for key, value in self.SCORE.items() :
            if key == action_[2]:
                reward = self.rewards['new_value'] + value[0]
                